# Interactive data visualizations

In [53]:
import numpy as np
import pandas as pd

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.sampledata.iris import flowers

output_notebook()

Loading BokehJS ...

In [2]:
flowers.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [4]:
print(type(flowers))

<class 'pandas.core.frame.DataFrame'>


In [23]:
flowers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [5]:
print(flowers['species'].unique())

['setosa' 'versicolor' 'virginica']


In [6]:
print(len(flowers))

150


In [9]:
plot = figure (x_axis_label = 'patal_length', y_axis_label = 'sepal_length')
plot.circle(flowers['petal_length'],flowers['sepal_length'])
show(plot)

In [10]:
type(plot)

bokeh.plotting.figure.Figure

In [11]:
plot = figure (x_axis_label = 'patal_length', y_axis_label = 'sepal_length')
plot.circle(flowers['petal_length'],flowers['sepal_length'])

GlyphRenderer(id='1135', ...)

In [12]:
from bokeh.models import CategoricalColorMapper

mapper = CategoricalColorMapper(
         factors = [ item for item in flowers['species'].unique() ],
         palette = ['red','green','blue'])

plot.circle('petal_length', 'sepal_length', source=flowers, color={'field': 'species', 'transform': mapper},legend_group = 'species')
plot.legend.location = 'top_left'
show(plot)

In [13]:
from bokeh.models import HoverTool
hover = HoverTool(tooltips=[('species name', '@species'), ('petal length', '@petal_length'), ('sepal length', '@sepal_length') ])
plot.add_tools(hover)
show(plot)

In [16]:
from bokeh.layouts import gridplot
plot1 = figure (x_axis_label = 'patal_length', y_axis_label = 'sepal_length')
plot1.circle('petal_length', 'sepal_length', source=flowers, color={'field': 'species', 'transform': mapper})
hover = HoverTool(tooltips=[('species name', '@species'), ('petal length', '@petal_length'), ('sepal length', '@sepal_length') ])
plot1.add_tools(hover)

plot2 = figure (x_axis_label = 'patal_length', y_axis_label = 'sepal_width')
plot2.circle('petal_length', 'sepal_width', source=flowers, color={'field': 'species', 'transform': mapper},legend_group = 'species')
plot2.add_tools(hover)

row = [plot1,plot2]

layout = gridplot([row])


In [17]:
show(layout)

In [60]:
from bokeh.layouts import column
from bokeh.models import Slider

from bokeh.sampledata.sea_surface_temperature import sea_surface_temperature

#From https://github.com/bokeh/bokeh/blob/2.2.3/examples/howto/server_embed/notebook_embed.ipynb

In [61]:
sea_surface_temperature.head()

,temperature
time,
2016-02-15 00:00:00+00:00,4.929
2016-02-15 00:30:00+00:00,4.887
2016-02-15 01:00:00+00:00,4.821
2016-02-15 01:30:00+00:00,4.837
2016-02-15 02:00:00+00:00,4.830


In [62]:
len(sea_surface_temperature)

19226

In [63]:
sea_surface_temperature.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 19226 entries, 2016-02-15 00:00:00+00:00 to 2017-03-22 00:00:00+00:00
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   temperature  19226 non-null  float64
dtypes: float64(1)
memory usage: 300.4 KB


In [66]:
def bkapp(doc):
    df = sea_surface_temperature.copy()
    source = ColumnDataSource(data=df)

    plot = figure(x_axis_type='datetime', y_range=(0, 25),
                  y_axis_label='Temperature (Celsius)',
                  title="Sea Surface Temperature at 43.18, -70.43")
    
    plot.line('time', 'temperature', source=source)

    def callback(attr, old, new):
        if new == 0:
            data = df
        else:
            data = df.rolling('{0}D'.format(new)).mean()
        source.data = ColumnDataSource.from_df(data)

    slider = Slider(start=0, end=30, value=0, step=1, title="Smoothing by N Days")
    
    slider.on_change('value', callback)

    doc.add_root(column(slider, plot))

In [65]:
show(bkapp)